# AIAK镜像打包自定义镜像

本文讲解如何基于aiak加速镜像打包自己的镜像

## 开发机镜像

```bash
# 使用NVIDIA PyTorch镜像
FROM registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release

# 确认容器内使用 root 用户
USER root

WORKDIR /workspace

# 安装Python包
RUN pip install jupyterlab -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    pip install jupyterlab-language-pack-zh-CN -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    pip install jupytext -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    curl -fsSL https://deb.nodesource.com/setup_16.x | bash - && \
    apt-get install -y nodejs && \
    mkdir -p /root/.install &&\
    cat <<EOF > /root/.install/enterpoint.sh
#!/bin/bash
echo "Starting JupyterLab..."
# 启动JupyterLab
JUPYTER_TOKEN=${JUPYTER_TOKEN}
echo "Jupyter Token is: ${JUPYTER_TOKEN}"
JUPYTERLAB_BASE_URL=${JUPYTERLAB_BASE_URL}
echo "JupyterLab Base URL is: ${JUPYTERLAB_BASE_URL}"
JUPYTER_PORT=8600
mkdir -p /workspace/.jupyter
nohup jupyter-lab --no-browser --NotebookApp.token="${JUPYTER_TOKEN}" --port=8600 --ip=0.0.0.0 --NotebookApp.base_url="${JUPYTERLAB_BASE_URL}" --notebook-dir="/workspace" > /workspace/.jupyter/jupyter.log 2>&1 &

if [ $? -ne 0 ]; then
    exit 1
fi

# 保持容器运行
while true; do
    sleep 60
done
EOF

# 检查/root/.install/enterpoint.sh文件是否创建成功并打印内容
RUN cat /root/.install/enterpoint.sh

# 打印环境变量
RUN env && echo "JUPYTER_TOKEN: ${JUPYTER_TOKEN}" && echo "JUPYTERLAB_BASE_URL: ${JUPYTERLAB_BASE_URL}"

RUN chmod +x /root/.install/enterpoint.sh && \
    mkdir -p /workspace/.jupyter && \
    cat <<EOF > /workspace/.jupyter/jupyter_notebook_config.py
import os
c = get_config()
c.NotebookApp.allow_remote_access = True
c.NotebookApp.allow_root = True
c.NotebookApp.terminado_settings = {"shell_command" : ["/bin/bash"]}
c.LabApp.base_url = os.environ.get("JUPYTERLAB_BASE_URL", "/jupyter")
c.LabApp.ip = "0.0.0.0"
c.LabApp.port = 8600
c.LabApp.terminado_settings = {"shell_command" : ["/bin/bash"]}
EOF

# 暴露端口：SSH 和 Jupyter
EXPOSE 8600
```

## 训练镜像

TBD